In [ ]:
prompt_template = """
You are a cybersecurity expert specializing in neural-symbolic defense systems for network intrusion detection. I need your assistance to adapt the symbolic component of our trained Transformer in reinforcement learning defense agent to enhance its performance in a new deployment environment.

## BACKGROUND AND PROBLEM FORMULATION

Our research involves a neural-symbolic defense agent for Network Intrusion Detection Systems (NIDS) focused on Distributed Denial of Service (DDoS) attacks. The agent architecture combines:
1. A neural component (RL agent) that processes network observations and selects actions
2. A symbolic component (rule-based program) that executes concrete defense operations based on the RL agent's action selection

While the agent performs optimally in its training environment, we observe significant performance degradation when deploying it in larger-scale environments. Rather than retraining the neural component (which would require substantial data collection, training time, and safety verification), we aim to adapt only the symbolic program component.

## ENVIRONMENT SPECIFICATIONS

### Current Simulator: NS3

### Training Environment Parameters:
- Attack agents (botnet nodes): {TRAIN_ATTACK_AGENTS_MIN}-{TRAIN_ATTACK_AGENTS_MAX}
- Legitimate clients: {TRAIN_LEGITIMATE_CLIENTS_MIN}-{TRAIN_LEGITIMATE_CLIENTS_MAX} 
- Servers/workstations in victim LAN: {TRAIN_SERVERS_MIN}-{TRAIN_SERVERS_MAX}
- Legitimate transmissions: {TRAIN_TRANSMISSIONS_MIN}-{TRAIN_TRANSMISSIONS_MAX}

### Deployment Environment Parameters:
- Attack agents (botnet nodes): {DEPLOY_ATTACK_AGENTS_MIN}-{DEPLOY_ATTACK_AGENTS_MAX} ({ATTACK_AGENTS_INCREASE_PERCENT}% increase)
- Legitimate clients: {DEPLOY_LEGITIMATE_CLIENTS_MIN}-{DEPLOY_LEGITIMATE_CLIENTS_MAX} ({LEGITIMATE_CLIENTS_INCREASE_PERCENT}% increase)
- Servers/workstations in victim LAN: {DEPLOY_SERVERS_MIN}-{DEPLOY_SERVERS_MAX} ({SERVERS_INCREASE_PERCENT}% increase)
- Legitimate transmissions: {DEPLOY_TRANSMISSIONS_MIN}-{DEPLOY_TRANSMISSIONS_MAX} ({TRANSMISSIONS_INCREASE_PERCENT}% increase)

## CURRENT SYMBOLIC PROGRAM

The neural component selects from four discrete actions {{a₀, a₁, a₂, a₃}}, with the following symbolic program implementation:

```C++
{CURRENT_SYMBOLIC_PROGRAM}
```

## PERFORMANCE METRICS AND DEGRADATION

We observe the following performance degradation as attack intensity increases in the deployment environment:

{PERFORMANCE_METRICS_TABLE}

The critical issues identified:
1. {ISSUE_1}
2. {ISSUE_2}
3. {ISSUE_3}
4. {ISSUE_4}

## YOUR TASK

Please propose specific modifications to our symbolic program to enhance its performance in the larger-scale deployment environment. Your recommendations should:

1. Maintain the overall interface with the neural component (four discrete actions)
2. Adapt the symbolic program's parameters, thresholds, and internal logic to handle significantly larger numbers of legitimate and malicious nodes
3. Address the identified bottlenecks in suspicious list management (especially for action a₂)
4. Reduce false positive rates while maintaining effective defense capabilities
5. Improve dynamic adjustment to varying network environment

For each proposed modification, please:
- Specify the exact code changes
- Explain the theoretical justification
- Discuss anticipated improvements in each performance metric
- Address potential trade-offs or limitations

Your guidance will directly inform our research on LLM-guided adaptation of symbolic programs for neural-symbolic cybersecurity agents.
"""

# Example usage with sample values
def format_prompt(
    # Training environment parameters
    train_attack_agents_min=5,
    train_attack_agents_max=10,
    train_legitimate_clients_min=15,
    train_legitimate_clients_max=30,
    train_servers_min=0,
    train_servers_max=10,
    train_transmissions_min=0,
    train_transmissions_max=10,
    
    # Deployment environment parameters
    deploy_attack_agents_min=20,
    deploy_attack_agents_max=60,
    deploy_legitimate_clients_min=100,
    deploy_legitimate_clients_max=300,
    deploy_servers_min=10,
    deploy_servers_max=30,
    deploy_transmissions_min=10,
    deploy_transmissions_max=30,
    
    # Current symbolic program code
    current_symbolic_program="""
    void ApplyAction(uint32_t action, std::map<Ipv4Address, SourceBehaviorStats>& sourceBehaviorMap) {
        // Step 1: compute suspicious score based on rules

        const double DECAY_RATE = 0.9;  // decay rate
        std::vector<std::pair<Ipv4Address, double>> activeSourceRanking;

        for (const auto& [addr, stats] : sourceBehaviorMap) {
            if (stats.isActive) {
                double dropRate = static_cast<double>(stats.totalDroppedPackets) / stats.totalTxPackets;
                double suspiciousScore = (dropRate > 0.1) 
                    ? dropRate * 0.8 + stats.activeRatio 
                    : stats.activeRatio;
                    
                activeSourceRanking.push_back({addr, suspiciousScore});
                
                // update the suspicious score for active sources
                auto it = SuspiciousList.find(addr);
                if (it != SuspiciousList.end()) {
                    it->second = suspiciousScore;  // update the score
                }
            } else {
                // decay the suspicious score for inactive sources
                auto it = SuspiciousList.find(addr);
                if (it != SuspiciousList.end()) {
                    it->second *= DECAY_RATE;  
                }
            }
        }

        // sort by suspicious score
        std::sort(activeSourceRanking.begin(), activeSourceRanking.end(),
                [](const auto& a, const auto& b) { return a.second > b.second; });

        // Step 2: symbolic actions
        std::string actionDescription;  
        switch (action) {
            case 0: {
                // observe
                actionDescription = "Observe (No action taken)";
                break;
            }

            case 1: {
                // add to Suspicious List
                int activeCount = activeSourceRanking.size();
                if (activeCount == 0) {
                    actionDescription = "Add to Suspicious List (No active sources to add)";
                    break;
                }

                int numToAdd = std::max(1, activeCount / 2);
                int count = 0;

                for (const auto& [addr, suspiciousScore] : activeSourceRanking) {
                    if (count >= numToAdd) break;

                    if (BlackList.find(addr) == BlackList.end() && 
                        SuspiciousList.find(addr) == SuspiciousList.end()) {
                        if (suspiciousScore < 0.2){
                            std::cout << "\n===== Push Failed due to low suspicious score (< 0.2): " << addr << " =====" << std::endl;
                            continue; 
                        }
                        SuspiciousList.insert({addr, suspiciousScore});
                        count++;
                    }
                }
                actionDescription = "Add to Suspicious List (" + std::to_string(count) + " sources added)";
                break;
            }

            case 2: {
                // remove from Suspicious List
                if (SuspiciousList.empty()) {
                    actionDescription = "Remove from Suspicious List (List is empty)";
                    break;
                }

                std::vector<std::pair<Ipv4Address, double>> suspiciousRanking(SuspiciousList.begin(), SuspiciousList.end());
                std::sort(suspiciousRanking.begin(), suspiciousRanking.end(),
                        [](const auto& a, const auto& b) { return a.second < b.second; });

                // get the address with the lowest suspicious score
                Ipv4Address toRemove = suspiciousRanking.front().first;
                double minSuspiciousScore = suspiciousRanking.front().second;

                // set a threshold for removing from the list
                const double suspiciousThreshold = 1.0;
                if (minSuspiciousScore > suspiciousThreshold) {
                    // refuse to remove
                    std::ostringstream oss;
                    oss << toRemove;  
                    actionDescription = "Failed to remove from Suspicious List (Address: " + oss.str() + ", Score too high: " + std::to_string(minSuspiciousScore) + ")";
                    testSuspiciousSuccess = 0;
                    break;
                }

                // remove from Suspicious List
                SuspiciousList.erase(toRemove);
                testSuspiciousSuccess = 1;
                std::ostringstream oss;
                oss << toRemove;  
                actionDescription = "Remove from Suspicious List (Address: " + oss.str() + ", Score: " + std::to_string(minSuspiciousScore) + ")";
                break;
            }

            case 3: {
                // promote to blacklist
                std::vector<Ipv4Address> toPromote;

                for (const auto& [addr, suspiciousScore] : SuspiciousList) {
                    auto& stats = sourceBehaviorMap[addr];
                    double dropRate = static_cast<double>(stats.totalDroppedPackets) / stats.totalTxPackets;
                    double timeDuration = stats.lastSeenTime - stats.firstSeenTime;
                    double TIME_THRESHOLD = 2.0; 

                    bool condition1 = stats.isActive &&
                                    timeDuration > TIME_THRESHOLD &&
                                    stats.activeRatio > 0.99;

                    bool condition2 = stats.isActive &&
                                    timeDuration > TIME_THRESHOLD &&
                                    dropRate > 0.6;

                    if (condition1 || condition2) {
                        toPromote.push_back(addr);
                        promoteBlackSuccess = 1;
                    } else {
                        promoteBlackSuccess = 0;
                    }
                }

                for (const auto& addr : toPromote) {
                    BlackList.insert(addr);
                    SuspiciousList.erase(addr);
                }
                
                ApplicationContainer newContainer;
                for (int k = 0; k < NUMBER_OF_BOTS; ++k) {
                    Ptr<Node> node = botNodes.Get(k);

                    // get Bot IP
                    Ipv4Address botIp = botInterfaces.GetAddress(k);
                    std::cout << "[INFO] Checking bot " << k << " with IP: " << botIp << std::endl;

                    // Check if the bot is in the blacklist
                    if (BlackList.find(botIp) != BlackList.end()) {
                        std::cout << "Blacklisted bot detected: " << botIp << std::endl;

                        // get OnOffApplication
                        Ptr<Application> app = node->GetApplication(0);
                        Ptr<OnOffApplication> onOffApp = DynamicCast<OnOffApplication>(app);

                        if (onOffApp) {
                            std::cout << "Stopping attack from bot: " << botIp << std::endl;
                            
                            // stop OnOffApplication
                            newContainer.Add(onOffApp);
                            newContainer.Stop(Seconds(Simulator::Now().GetSeconds() + 0.1));

                            std::cout << "Attack stopped for bot with IP " << botIp << std::endl;
                        } else {
                            std::cout << "Failed to retrieve OnOffApplication for bot: " << botIp << std::endl;
                        }
                    }
                }



                actionDescription = "Promote to Blacklist (" + std::to_string(toPromote.size()) + " sources promoted)";
                break;
            }

            default: {
                actionDescription = "Invalid Action";
                break;
            }
        }

        // Step 3: print
        std::cout << "\n===== Action Taken: " << actionDescription << " =====" << std::endl;

        std::cout << "\n===== Current Suspicious List =====" << std::endl;
        for (const auto& [addr, score] : SuspiciousList) {
            std::cout << "Address: " << addr 
                    << ", Suspicious Score: " << score << std::endl;
        }

        std::cout << "\n===== Current Blacklist =====" << std::endl;
        for (const auto& addr : BlackList) {
            std::cout << "Address: " << addr << std::endl;
        }
        std::cout << "====================================\n" << std::endl;
    }

    """,
    
    # Performance metrics table
    performance_metrics_table="""| Attack Intensity (Bot Nodes) | 20    | 30    | 40    | 50    | 60    |
                                |------------------------------|-------|-------|-------|-------|-------|
                                | Network Uptime Rate (%)      | 68.63 | 50.98 | 35.29 | 29.41 | 20.69 |
                                | Malicious Node Detection (%) | 100.0 | 100.0 | 100.0 | 98.00 | 85.00 |
                                | False Blocking Rate (%)      | 5.50  | 18.00 | 32.50 | 45.00 | 59.25 |""",
    
    # Critical issues
    issue_1="While malicious node detection remains robust, network uptime deteriorates rapidly",
    issue_2="False blocking rate escalates significantly with increased attack intensity",
    issue_3="The symbolic action framework fails to provide timely defensive responses under high-pressure conditions",
    issue_4="The suspicious list management becomes a critical bottleneck"
):
    # Calculate increase percentages
    attack_agents_increase = int(((deploy_attack_agents_max/train_attack_agents_max) - 1) * 100)
    legitimate_clients_increase = int(((deploy_legitimate_clients_max/train_legitimate_clients_max) - 1) * 100)
    servers_increase = int(((deploy_servers_max/train_servers_max) - 1) * 100) if train_servers_max > 0 else 200
    transmissions_increase = int(((deploy_transmissions_max/train_transmissions_max) - 1) * 100) if train_transmissions_max > 0 else 200
    
    # Format the prompt with all parameters
    formatted_prompt = prompt_template.format(
        TRAIN_ATTACK_AGENTS_MIN=train_attack_agents_min,
        TRAIN_ATTACK_AGENTS_MAX=train_attack_agents_max,
        TRAIN_LEGITIMATE_CLIENTS_MIN=train_legitimate_clients_min,
        TRAIN_LEGITIMATE_CLIENTS_MAX=train_legitimate_clients_max,
        TRAIN_SERVERS_MIN=train_servers_min,
        TRAIN_SERVERS_MAX=train_servers_max,
        TRAIN_TRANSMISSIONS_MIN=train_transmissions_min,
        TRAIN_TRANSMISSIONS_MAX=train_transmissions_max,
        
        DEPLOY_ATTACK_AGENTS_MIN=deploy_attack_agents_min,
        DEPLOY_ATTACK_AGENTS_MAX=deploy_attack_agents_max,
        DEPLOY_LEGITIMATE_CLIENTS_MIN=deploy_legitimate_clients_min,
        DEPLOY_LEGITIMATE_CLIENTS_MAX=deploy_legitimate_clients_max,
        DEPLOY_SERVERS_MIN=deploy_servers_min,
        DEPLOY_SERVERS_MAX=deploy_servers_max,
        DEPLOY_TRANSMISSIONS_MIN=deploy_transmissions_min,
        DEPLOY_TRANSMISSIONS_MAX=deploy_transmissions_max,
        
        ATTACK_AGENTS_INCREASE_PERCENT=attack_agents_increase,
        LEGITIMATE_CLIENTS_INCREASE_PERCENT=legitimate_clients_increase,
        SERVERS_INCREASE_PERCENT=servers_increase,
        TRANSMISSIONS_INCREASE_PERCENT=transmissions_increase,
        
        CURRENT_SYMBOLIC_PROGRAM=current_symbolic_program,
        PERFORMANCE_METRICS_TABLE=performance_metrics_table,
        
        ISSUE_1=issue_1,
        ISSUE_2=issue_2,
        ISSUE_3=issue_3,
        ISSUE_4=issue_4
    )
    
    return formatted_prompt



In [ ]:
final_prompt = format_prompt()
print(final_prompt)


You are a cybersecurity expert specializing in neural-symbolic defense systems for network intrusion detection. I need your assistance to adapt the symbolic component of our trained Transformer in reinforcement learning defense agent to enhance its performance in a new deployment environment.

## BACKGROUND AND PROBLEM FORMULATION

Our research involves a neural-symbolic defense agent for Network Intrusion Detection Systems (NIDS) focused on Distributed Denial of Service (DDoS) attacks. The agent architecture combines:
1. A neural component (RL agent) that processes network observations and selects actions
2. A symbolic component (rule-based program) that executes concrete defense operations based on the RL agent's action selection

While the agent performs optimally in its training environment, we observe significant performance degradation when deploying it in larger-scale environments. Rather than retraining the neural component (which would require substantial data collection, tra